In [1]:
import os
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader
import foolbox as fb
from captum.attr import IntegratedGradients
from openood.evaluation_api.datasets import DATA_INFO
from openood.evaluation_api.preprocessor import get_default_preprocessor, default_preprocessing_dict
from openood.datasets.imglist_dataset import ImglistDataset
from openood.networks import ResNet18_32x32, ResNet18_224x224

In [2]:
def get_dataloader(id_name, data_root = 'data/', split = 'test'):
    preprocessor = get_default_preprocessor(id_name)
    test_standard_preprocessor = get_default_preprocessor(id_name)
    data_info = DATA_INFO[id_name]
    norm = default_preprocessing_dict[id_name]['normalization']
    dataset = ImglistDataset(
        name='_'.join((id_name, split)),
        imglist_pth=os.path.join(data_root,
                                 data_info['id'][split]['imglist_path']),
        data_dir=os.path.join(data_root,
                              data_info['id'][split]['data_dir']),
        num_classes=data_info['num_classes'],
        preprocessor=preprocessor,
        data_aux_preprocessor=test_standard_preprocessor)
    dataloader = DataLoader(dataset, batch_size=64 if 'net' in id_name else 200,shuffle=False, num_workers=8)
    mean, std = [torch.tensor(n, device='cuda').reshape(-1, 1, 1) for n in norm]
    return dataloader, (mean, std)

def get_nets(id_name, data_root='results/'):
    NUM_CLASSES = {'cifar10': 10, 'cifar100': 100, 'imagenet200': 200}
    MODEL = {
        'cifar10': ResNet18_32x32,
        'cifar100': ResNet18_32x32,
        'imagenet200': ResNet18_224x224,
    }
    names = {'cifar10': 'cifar10_resnet18_32x32_base_e100_lr0.1_default',
         'cifar100': 'cifar100_resnet18_32x32_base_e100_lr0.1_default',
         'imagenet200': 'imagenet200_resnet18_224x224_base_e90_lr0.1_default'}
    nets = []
    for i in range(3):
        net = MODEL[id_name](num_classes=NUM_CLASSES[id_name]) #ResNet18_32x32(num_classes=10)
        net.load_state_dict(torch.load(os.path.join(f'{data_root}{names[id_name]}/s{i}/', 'best.ckpt'), map_location='cpu'))
        net.cuda()
        net.eval()
        nets.append(net)
    return nets

def get_fb_model(net, mean, std):
    bounds = (((0-mean)/std).min().item(), ((1-mean)/std).max().item())
    return fb.PyTorchModel(net, bounds=bounds)

def get_advs(dataloader, fmodel, attack, mean, std, eps=0.3):
    full_advs = []
    for data_dict in tqdm(dataloader):
        x, y = data_dict['data'].cuda(), data_dict['label'].cuda()
        raw, advs, success = attack(fmodel, x, y, epsilons=eps)
        #print(success.sum())
        all_success = torch.stack([torch.logical_and(n(x).argmax(1) == y, n(advs).argmax(1) != y) for n in nets]).sum(0) == 3
        #print(all_success.sum())
        full_advs.append(((advs[all_success]*std + mean)*255).to('cpu', torch.uint8))
    return torch.concatenate(full_advs).permute(0, 2, 3, 1).numpy()

def integrated_gradient_method(net, input_image, rate = 0.7, minmax=(0, 1)):
    img = input_image.clone()
    integrated_grad = IntegratedGradients(net)
    baseline = torch.zeros_like(img)
    initial_class = get_output_class(net, img)
    pred_class = initial_class
    while initial_class == pred_class:
        net.zero_grad()
        ig_result = integrated_grad.attribute(img, target=initial_class, baselines=baseline)
        amax = np.unravel_index(ig_result.argmax().item(), ig_result.shape)
        img[amax] = img[amax] * rate
        pred_class = get_output_class(net, img)
    return img

In [3]:
import eagerpy as ep
from foolbox.attacks.base import get_criterion
from foolbox.attacks.base import raise_if_kwargs
from foolbox.attacks.base import verify_input_bounds
from foolbox.criteria import Misclassification, TargetedMisclassification

class IGM(fb.attacks.LinfProjectedGradientDescentAttack):
    def __init__(
        self,
        *,
        rel_stepsize: float = 0.01 / 0.3,
        abs_stepsize = None,
        steps: int = 10,
        random_start: bool = True,
    ):
        super().__init__(
            rel_stepsize=rel_stepsize,
            abs_stepsize=abs_stepsize,
            steps=steps,
            random_start=random_start,
        )
    #https://github.com/bethgelab/foolbox/blob/master/foolbox/attacks/gradient_descent_base.py
    def run(
        self,
        model,
        inputs,
        criterion ,
        *,
        epsilon: float,
        **kwargs,
    ):
        raise_if_kwargs(kwargs)
        x0, restore_type = ep.astensor_(inputs)
        criterion_ = get_criterion(criterion)
        del inputs, criterion, kwargs

        verify_input_bounds(x0, model)

        # perform a gradient ascent (targeted attack) or descent (untargeted attack)
        if isinstance(criterion_, Misclassification):
            gradient_step_sign = 1.0
            classes = criterion_.labels
        elif hasattr(criterion_, "target_classes"):
            gradient_step_sign = -1.0
            classes = criterion_.target_classes
        else:
            raise ValueError("unsupported criterion")
            
        # MOD HERE
        nint = 8
        scales_int = torch.linspace(0, 1, nint, device='cuda').reshape(-1, 1, 1, 1, 1)
        classes = ep.astensor(classes.raw.repeat(nint))
        ##
        loss_fn = self.get_loss_fn(model, classes)

        if self.abs_stepsize is None:
            stepsize = self.rel_stepsize * epsilon
        else:
            stepsize = self.abs_stepsize

        optimizer = self.get_optimizer(x0, stepsize)

        if self.random_start:
            x = self.get_random_start(x0, epsilon)
            x = ep.clip(x, *model.bounds)
        else:
            x = x0

        for _ in range(self.steps):
            x_exp = (scales_int * x).flatten(0, 1)                       # MOD HERE
            _, gradients = self.value_and_grad(loss_fn, x_exp)
            gradients = ep.astensor(gradients.raw.unflatten(0, (nint, -1)).sum(0) / nint) # MOD HERE
            gradients = self.normalize(gradients, x=x, bounds=model.bounds)
            x = x + gradient_step_sign * optimizer(gradients)
            x = self.project(x, x0, epsilon)
            x = ep.clip(x, *model.bounds)

        return restore_type(x)

In [4]:
attacks = {
    'fgsm': fb.attacks.FGSM(),
    'pgd': fb.attacks.PGD(),
    'igm': IGM(),
    'iter': fb.attacks.LinfBasicIterativeAttack()
}

In [90]:
for id_name in ['cifar10', 'cifar100', 'imagenet200']:
    for attack_name, attack in attacks.items():
        print(f'EVALUATING IN {id_name}, attack: {attack_name}')
        dataloader, mean_std = get_dataloader(id_name)
        nets = get_nets(id_name)
        fmodel = get_fb_model(nets[0], *mean_std)
        advs = get_advs(dataloader, fmodel, attack, *mean_std)
        np.save(f'data/benchmark_imglist/{id_name}/{attack_name}.npy', advs)

EVALUATING IN cifar10, attack: igm


  0%|                                                                                                          | 0/45 [00:00<?, ?it/s]


AttributeError: 'PyTorchTensor' object has no attribute 'repeat'

In [4]:
attacks = {
    'igm': IGM(),
    'iter': fb.attacks.LinfBasicIterativeAttack()
}
for id_name in ['imagenet200']:
    for attack_name, attack in attacks.items():
        print(f'EVALUATING IN {id_name}, attack: {attack_name}')
        dataloader, mean_std = get_dataloader(id_name)
        nets = get_nets(id_name)
        fmodel = get_fb_model(nets[0], *mean_std)
        advs = get_advs(dataloader, fmodel, attack, *mean_std)
        np.save(f'data/benchmark_imglist/{id_name}/{attack_name}.npy', advs)

EVALUATING IN imagenet200, attack: igm


 99%|██████████████████████████████████████████████████████████████████████████████████████████████▎| 140/141 [09:04<00:03,  3.91s/it]/home/gabriel/.pyenv/versions/3.10.12/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████████████████████████████████████████████████████████| 141/141 [09:06<00:00,  3.88s/it]


EVALUATING IN imagenet200, attack: iter


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 141/141 [01:30<00:00,  1.55it/s]


In [6]:
a, b = torch.rand(20, 20).max(0)
a, b

(tensor([0.9524, 0.9230, 0.9951, 0.9757, 0.9724, 0.9918, 0.9626, 0.9823, 0.9707,
         0.9622, 0.9664, 0.9999, 0.9881, 0.9320, 0.9940, 0.8786, 0.9919, 0.9533,
         0.8107, 0.9664]),
 tensor([ 5, 15, 19, 16,  6,  7, 15, 14, 14, 17,  2, 15, 14, 17,  0,  7,  4, 13,
         16, 11]))

In [69]:
%debug

> /home/gabriel/.pyenv/versions/3.10.12/lib/python3.10/site-packages/eagerpy/tensor/pytorch.py(468)crossentropy()
    466             raise ValueError("crossentropy only supported for 2D logits tensors")
    467         if self.shape[:1] != labels.shape:
--> 468             raise ValueError("labels must be 1D and must match the length of logits")
    469         return type(self)(
    470             torch.nn.functional.cross_entropy(self.raw, labels.raw, reduction="none")



ipdb>  up


> /home/gabriel/.pyenv/versions/3.10.12/lib/python3.10/site-packages/eagerpy/framework.py(325)crossentropy()
    323 
    324 def crossentropy(logits: TensorType, labels: TensorType) -> TensorType:
--> 325     return logits.crossentropy(labels)
    326 
    327 



ipdb>  labels


PyTorchTensor(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'))


...

ipdb>  up


> /tmp/ipykernel_75979/528264053.py(69)run()
     67         for _ in range(self.steps):
     68             x_exp = (scales_int * x).flatten(0, 1)
---> 69             _, gradients = self.value_and_grad(loss_fn, x_exp)
     70             gradients = gradients.unflatten(0, (nint, -1)).sum(0) / nint
     71             gradients = self.normalize(gradients, x=x, bounds=model.bounds)



ipdb>  criterion_


Misclassification(PyTorchTensor(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')))


ipdb>  r


In [63]:
((x * std + mean)*255).min().to('cpu', torch.uint8)

tensor(0, dtype=torch.uint8)

In [21]:
datas = next(iter(dataloader))
x, y = datas['data'].cuda(), datas['label'].cuda()

In [4]:
fmodel = fb.PyTorchModel(net, bounds=(-2, 2.13))
attack = fb.attacks.FGSM()

In [7]:
fb.utils.accuracy(fmodel, datas['data'].cuda(), datas['label'].cuda())

0.9199999570846558

In [34]:
raw, advs, success = attack(fmodel, datas['data'].cuda(), datas['label'].cuda(), epsilons=0.3)

In [28]:
(torch.stack([n(x).argmax(1) == y for n in nets]).sum(0) == 3).sum()

tensor(178, device='cuda:0')

In [31]:
(torch.stack([torch.logical_and(n(x).argmax(1) == y, n(advs).argmax(1) != y) for n in nets]).sum(0) == 3).sum()

tensor(157, device='cuda:0')

  0%|                                                    | 0/45 [00:00<?, ?it/s]/home/gabriel/.pyenv/versions/3.10.12/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|███████████████████████████████████████████| 45/45 [00:06<00:00,  6.73it/s]


In [70]:
np.array([len(n) for n in names_advs]).sum()

5564

In [6]:
np.save('data/benchmark_imglist/cifar10/fgsm.npy', torch.concatenate(full_advs).permute(0, 2, 3, 1).numpy())

In [7]:
fmodel = fb.PyTorchModel(net, bounds=(-2, 2.13))
attack = fb.attacks.PGD()

In [8]:
full_advs = []
names_advs = []
for data_dict in tqdm(dataloader):
    x, y = data_dict['data'].cuda(), data_dict['label'].cuda()
    raw, advs, success = attack(fmodel, x, y, epsilons=0.3)
    all_success = torch.stack([torch.logical_and(n(x).argmax(1) == y, n(advs).argmax(1) != y) for n in nets]).sum(0) == 3
    full_advs.append(((advs[all_success]*std + mean)*255).to('cpu', torch.uint8))
    names_advs.append(np.array(data_dict['image_name'])[all_success.cpu().numpy()])

100%|███████████████████████████████████████████| 45/45 [01:18<00:00,  1.73s/it]


In [83]:
np.array([len(n) for n in names_advs]).sum()

7869

In [9]:
np.save('data/benchmark_imglist/cifar10/pgd.npy', torch.concatenate(full_advs).permute(0, 2, 3, 1).numpy())

In [71]:
3*224*244*10_000 / 10**6

1639.68

In [45]:
np.array(datas['image_name'])[[0, 3, 2]]

array(['cifar10/test/airplane/0298.png', 'cifar10/test/airplane/0515.png',
       'cifar10/test/airplane/0501.png'], dtype='<U30')

In [31]:
datas.keys()

dict_keys(['image_name', 'data', 'data_aux', 'label', 'soft_label', 'index', 'pseudo'])

In [29]:
(np.array([0]*3) - np.array([0.4914, 0.4822, 0.4465])) / np.array([0.2470, 0.2435, 0.2616])

array([-1.98947368, -1.98028747, -1.70680428])

In [1]:
0.3/4*255

19.125

In [14]:
datas

{'image_name': ['cifar10/test/airplane/0298.png',
  'cifar10/test/airplane/0267.png',
  'cifar10/test/airplane/0501.png',
  'cifar10/test/airplane/0515.png',
  'cifar10/test/airplane/0273.png',
  'cifar10/test/airplane/0529.png',
  'cifar10/test/airplane/0924.png',
  'cifar10/test/airplane/0930.png',
  'cifar10/test/airplane/0703.png',
  'cifar10/test/airplane/0065.png',
  'cifar10/test/airplane/0071.png',
  'cifar10/test/airplane/0059.png',
  'cifar10/test/airplane/0878.png',
  'cifar10/test/airplane/0688.png',
  'cifar10/test/airplane/0844.png',
  'cifar10/test/airplane/0677.png',
  'cifar10/test/airplane/0105.png',
  'cifar10/test/airplane/0663.png',
  'cifar10/test/airplane/0139.png',
  'cifar10/test/airplane/0887.png',
  'cifar10/test/airplane/0313.png',
  'cifar10/test/airplane/0475.png',
  'cifar10/test/airplane/0461.png',
  'cifar10/test/airplane/0307.png',
  'cifar10/test/airplane/0449.png',
  'cifar10/test/airplane/0460.png',
  'cifar10/test/airplane/0306.png',
  'cifar10/tes